In [ ]:
# Manysig SNR循环实验 ResNet 6TX
# === 导入必要库 ===
from joblib import load
import pandas as pd
import numpy as np
import os
from data_utilities import *
import cv2
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import gc
from tqdm.auto import tqdm
from collections import defaultdict
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Subset
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import seaborn as sns
from datetime import datetime

# === 数据加载和预处理 ===
dataset_name = 'ManySig'
dataset_path='../ManySig.pkl/'

compact_dataset = load_compact_pkl_dataset(dataset_path,dataset_name)

print("数据集发射机数量：",len(compact_dataset['tx_list']),"具体为：",compact_dataset['tx_list'])
print("数据集接收机数量：",len(compact_dataset['rx_list']),"具体为：",compact_dataset['rx_list'])
print("数据集采集天数：",len(compact_dataset['capture_date_list']),"具体为：",compact_dataset['capture_date_list'])

tx_list = compact_dataset['tx_list']
rx_list = compact_dataset['rx_list']
capture_date_list = compact_dataset['capture_date_list']

n_tx = len(tx_list)
n_rx = len(rx_list)

train_dates = ['2021_03_15']
test_dates  = ['2021_03_01']
equalized = 0

X_train, y_train, X_test, y_test = preprocess_dataset_for_classification_cross_date(
    compact_dataset, tx_list, rx_list, train_dates, test_dates, max_sig=None, equalized=equalized)


print("训练集所选日期：",train_dates, "测试集所选日期：", test_dates)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test  shape:", X_test.shape) 
print("y_test  shape:", y_test.shape)

# === 信号处理参数 ===
fs = 20e6
fc = 2.4e9
v = 120
Add_noise = True
Add_doppler = True

def compute_doppler_shift(v, fc):
    c = 3e8
    v = v / 3.6
    return (v / c) * fc

fd = compute_doppler_shift(v, fc)

def add_doppler_shift(signal, fd, fs):
    num_samples = signal.shape[-1]
    t = np.arange(num_samples) / fs
    doppler_phase = np.exp(1j * 2 * np.pi * fd * t)
    return signal * doppler_phase

def measure_snr(clean_signal, noisy_signal):
    signal_power = np.mean(np.abs(clean_signal) ** 2)
    noise = noisy_signal - clean_signal
    noise_power = np.mean(np.abs(noise) ** 2)
    if noise_power == 0:
        return float('inf')
    return 10 * np.log10(signal_power / noise_power)

def add_complex_awgn(signal, snr_db):
    signal_power = np.mean(np.abs(signal) ** 2)
    snr_linear = 10 ** (snr_db / 10)
    noise_power = signal_power / snr_linear
    noise_std = np.sqrt(noise_power / 2)
    noise = np.random.normal(0, noise_std, signal.shape) + 1j*np.random.normal(0, noise_std, signal.shape)
    return signal + noise, noise

def preprocess_iq_data(data_real_imag, snr_db=None, fd=None, fs=None, add_noise=True, add_doppler=True, verify_snr=True):
    if add_noise and snr_db is None:
        raise ValueError("当add_noise=True时，必须提供snr_db参数")
    if add_doppler and (fd is None or fs is None):
        raise ValueError("当add_doppler=True时，必须提供fd和fs参数")
    data_complex = data_real_imag[...,0] + 1j*data_real_imag[...,1]
    processed = []
    snr_measured_list = []
    for i, sig in enumerate(data_complex):
         # 原始信号归一化
        sig = sig / (np.sqrt(np.mean(np.abs(sig)**2)) + 1e-12)
        current_sig = sig.copy()
        if add_doppler:
            current_sig = add_doppler_shift(current_sig, fd, fs)
        if add_noise:
            noisy_sig, _ = add_complex_awgn(current_sig, snr_db)
            current_sig = noisy_sig
            if verify_snr:
                measured_snr = measure_snr(sig, noisy_sig)
                snr_measured_list.append(measured_snr)
        processed.append(current_sig)
    processed = np.array(processed)
    processed_real_imag = np.stack([processed.real, processed.imag], axis=-1)
    return processed_real_imag, None

# === ResNet 1D 模型定义 ===
class BasicBlock1D(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1, downsample=None, dropout=0.0):
        super().__init__()
        self.conv1 = nn.Conv1d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm1d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(p=dropout)
        self.conv2 = nn.Conv1d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(planes)
        self.downsample = downsample
    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.dropout(out)
        out = self.bn2(self.conv2(out))
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        return self.relu(out)

class ResNet18_1D(nn.Module):
    def __init__(self, num_classes=10, in_planes=64, dropout=0.0):
        super().__init__()
        self.in_planes = in_planes
        self.conv1 = nn.Conv1d(2, in_planes, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm1d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(64, 2, stride=1, dropout=dropout)
        self.layer2 = self._make_layer(128, 2, stride=2, dropout=dropout)
        self.layer3 = self._make_layer(256, 2, stride=2, dropout=dropout)
        self.layer4 = self._make_layer(512, 2, stride=2, dropout=dropout)
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(512, num_classes)
    def _make_layer(self, planes, blocks, stride, dropout):
        downsample = None
        if stride != 1 or self.in_planes != planes:
            downsample = nn.Sequential(
                nn.Conv1d(self.in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(planes)
            )
        layers = [BasicBlock1D(self.in_planes, planes, stride, downsample, dropout)]
        self.in_planes = planes
        for _ in range(1, blocks):
            layers.append(BasicBlock1D(self.in_planes, planes, dropout=dropout))
        return nn.Sequential(*layers)
    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.squeeze(-1)
        return self.fc(x)

# === 训练超参数 ===
batch_size   = 64
num_epochs   = 200
learning_rate = 1e-4
weight_decay = 1e-3
in_planes    = 64
dropout      = 0.5
patience     = 5
n_splits     = 5
num_classes  = len(np.unique(y_train))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def compute_grad_norm(model):
    total_norm = 0.0
    for p in model.parameters():
        if p.grad is not None:
            total_norm += (p.grad.data.norm(2).item() ** 2)
    return total_norm ** 0.5

def moving_average(x, w=5):
    return np.convolve(x, np.ones(w), 'valid') / w

# === SNR 循环 ===
snr_list = list(range(-25, -45, -5))  # 20,15,...,-40
all_results = {}

for SNR_dB in snr_list:
    print(f"\n===== 开始 SNR={SNR_dB} dB 实验 =====")

    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    script_name = "wisig_time"
    folder_name = f"{timestamp}_{script_name}_SNR{SNR_dB}dB_fd{int(fd)}_classes_{num_classes}_ResNet18"
    save_folder = os.path.join(os.getcwd(), "training_results", folder_name)
    os.makedirs(save_folder, exist_ok=True)
    
    results_file = os.path.join(save_folder, "results.txt")
    with open(results_file, "w") as f:
        f.write(f"=== Experiment Summary ===\n")
        f.write(f"Timestamp: {timestamp}\n")
        f.write(f"Total Classes: {num_classes}\n")
        f.write(f"SNR: {SNR_dB} dB\n")
        f.write(f"fd (Doppler shift): {fd} Hz\n")
        f.write(f"equalized: {equalized}\n")
        f.write(f"训练集所选日期: {train_dates}")
        f.write(f"测试集所选日期：{test_dates}")

    # === 数据处理 ===
    X_train_processed, _ = preprocess_iq_data(
        X_train, snr_db=SNR_dB, fd=fd, fs=fs, add_noise=Add_noise, add_doppler=Add_doppler, verify_snr=False
    )
    X_test_processed, _ = preprocess_iq_data(
        X_test, snr_db=SNR_dB, fd=fd, fs=fs, add_noise=Add_noise, add_doppler=Add_doppler, verify_snr=False
    )

    train_dataset = TensorDataset(torch.tensor(X_train_processed,dtype=torch.float32),
                                  torch.tensor(y_train,dtype=torch.long))
    test_dataset = TensorDataset(torch.tensor(X_test_processed,dtype=torch.float32),
                                 torch.tensor(y_test,dtype=torch.long))
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    fold_results = []
    test_results = []
    avg_grad_norms_per_fold = []

    for fold, (train_idx, val_idx) in enumerate(kfold.split(train_dataset)):
        print(f"\n=== Fold {fold+1}/{n_splits} ===")
        train_subset = Subset(train_dataset, train_idx)
        val_subset = Subset(train_dataset, val_idx)
        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, drop_last=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, drop_last=True)

        model = ResNet18_1D(num_classes=num_classes, in_planes=in_planes, dropout=dropout).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

        train_losses, val_losses = [], []
        train_accuracies, val_accuracies = [], []
        grad_norms = []
        best_val_loss = float('inf')
        patience_counter = 0

        for epoch in range(num_epochs):
            model.train()
            running_train_loss, correct_train, total_train = 0.0, 0, 0
            batch_grad_norms = []
            with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as tepoch:
                for inputs, labels in tepoch:
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    grad_norm = compute_grad_norm(model)
                    batch_grad_norms.append(grad_norm)
                    optimizer.step()
                    running_train_loss += loss.item()
                    _, predicted = torch.max(outputs, 1)
                    total_train += labels.size(0)
                    correct_train += (predicted == labels).sum().item()
                    tepoch.set_postfix(loss=running_train_loss/len(train_loader),
                                       accuracy=100*correct_train/total_train,
                                       grad_norm=grad_norm)

            epoch_train_loss = running_train_loss/len(train_loader)
            train_losses.append(epoch_train_loss)
            train_accuracies.append(100*correct_train/total_train)
            avg_grad_norm = np.mean(batch_grad_norms)
            grad_norms.append(avg_grad_norm)

            # 验证
            model.eval()
            running_val_loss, correct_val, total_val = 0.0, 0, 0
            with torch.no_grad():
                for val_inputs, val_labels in val_loader:
                    val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
                    val_outputs = model(val_inputs)
                    val_loss = criterion(val_outputs, val_labels)
                    running_val_loss += val_loss.item()
                    _, val_predicted = torch.max(val_outputs, 1)
                    total_val += val_labels.size(0)
                    correct_val += (val_predicted == val_labels).sum().item()
            epoch_val_loss = running_val_loss / len(val_loader)
            val_losses.append(epoch_val_loss)
            val_accuracies.append(100*correct_val/total_val)

            with open(results_file,'a') as f:
                f.write(f"Fold{fold+1} Epoch{epoch+1} | TrainAcc={train_accuracies[-1]:.2f}% | ValAcc={val_accuracies[-1]:.2f}% | "
                        f"TrainLoss={train_losses[-1]:.4f} | ValLoss={val_losses[-1]:.4f} | AvgGrad={avg_grad_norm:.4f}\n")


            if epoch_val_loss < best_val_loss:
                best_val_loss = epoch_val_loss
                patience_counter = 0
            else:
                patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping")
                break
            scheduler.step()

        fold_results.append(max(val_accuracies))
        avg_grad_norms_per_fold.append(grad_norms)

        # 绘图
        plt.figure()
        plt.plot(train_losses, label='Train Loss')
        plt.plot(val_losses, label='Val Loss')
        plt.plot(moving_average(train_losses), label='Train Loss Smooth', linestyle='--')
        plt.plot(moving_average(val_losses), label='Val Loss Smooth', linestyle='--')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title(f'Fold {fold+1} Loss Curve')
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(save_folder, f"fold_{fold+1}_loss_curve.png"))
        plt.close()

        plt.figure()
        plt.plot(grad_norms, label='Grad Norm')
        plt.xlabel('Epoch')
        plt.ylabel('Grad Norm')
        plt.title(f'Fold {fold+1} Grad Norm')
        plt.grid(True)
        plt.legend()
        plt.savefig(os.path.join(save_folder, f"fold_{fold+1}_grad_norm.png"))
        plt.close()

        # 测试集评估
        model.eval()
        test_preds, test_true = [], []
        with torch.no_grad():
            for test_inputs, test_labels in test_loader:
                test_inputs, test_labels = test_inputs.to(device), test_labels.to(device)
                test_outputs = model(test_inputs)
                _, predicted = torch.max(test_outputs, 1)
                test_preds.extend(predicted.cpu().numpy())
                test_true.extend(test_labels.cpu().numpy())
        test_preds = np.array(test_preds)
        test_true = np.array(test_true)
        test_accuracy = 100*np.sum(test_preds==test_true)/len(test_true)
        test_results.append(test_accuracy)

        print(f"Fold {fold+1} Test Accuracy: {test_accuracy:.2f}%")
        with open(results_file, "a") as f:
            f.write(f"Fold {fold+1} Test Acc: {test_accuracy:.2f}%\n")

        cm = confusion_matrix(test_true, test_preds)
        plt.figure(figsize=(10,8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title(f'Fold {fold+1} Confusion Matrix')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.savefig(os.path.join(save_folder, f"fold_{fold+1}_confusion_matrix.png"))
        plt.close()

    avg_val = np.mean(fold_results)
    avg_test = np.mean(test_results)
    with open(results_file, "a") as f:
        f.write(f"\n=== SNR={SNR_dB} Summary ===\n")
        f.write(f"Avg Val Acc: {avg_val:.2f}%, Avg Test Acc: {avg_test:.2f}%\n")
    print(f"SNR={SNR_dB} 完成: Avg Val={avg_val:.2f}%, Avg Test={avg_test:.2f}%")
    all_results[SNR_dB] = save_folder

print("\n===== 所有 SNR 实验完成 =====")
for snr, folder in all_results.items():
    print(f"SNR={snr} 结果保存在: {folder}")


In [ ]:
# 初始版 ManySig Transfomer 6TX
from joblib import load
import pandas as pd
import numpy as np
import os
from  data_utilities import *
import cv2  # OpenCV 用于调整图像大小和颜色处理
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import gc  # 引入垃圾回收模块
from tqdm.auto import tqdm  # 自动适配环境 导入tqdm进度条库
from collections import defaultdict

dataset_name = 'ManySig'
dataset_path='../ManySig.pkl/'

compact_dataset = load_compact_pkl_dataset(dataset_path,dataset_name)

print("数据集发射机数量：",len(compact_dataset['tx_list']),"具体为：",compact_dataset['tx_list'])
print("数据集接收机数量：",len(compact_dataset['rx_list']),"具体为：",compact_dataset['rx_list'])
print("数据集采集天数：",len(compact_dataset['capture_date_list']),"具体为：",compact_dataset['capture_date_list'])


tx_list = compact_dataset['tx_list']
rx_list = compact_dataset['rx_list']
equalized = 0
capture_date_list = compact_dataset['capture_date_list']


n_tx = len(tx_list)
n_rx = len(rx_list)
print(n_tx,n_rx)


train_dates = ['2021_03_15']  # 设定训练日期
test_dates  = ['2021_03_01']  # 设定测试日期
X_train, y_train, X_test, y_test = preprocess_dataset_for_classification_cross_date(
    compact_dataset, tx_list, rx_list, train_dates, test_dates, max_sig=None, equalized = equalized)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test  shape:", X_test.shape)
print("y_test  shape:", y_test.shape)

import numpy as np

# === 参数设置 ===
SNR_dB = 15           # 信噪比
fs = 20e6             # 采样率 (Hz)
fc = 2.4e9            # 载波频率 (Hz)
v = 120               # 速度 (km/h)
Add_noise = True     # 是否添加噪声
Add_doppler = True   # 是否添加多普勒频移

# === 多普勒频移计算 ===
def compute_doppler_shift(v, fc):
    c = 3e8  # 光速
    v = v / 3.6 # 转换为m/s
    return (v / c) * fc

fd = compute_doppler_shift(v, fc)
print(f"[INFO] 多普勒频移 fd = {fd:.2f} Hz")

# === 多普勒变换 ===
def add_doppler_shift(signal, fd, fs):
    num_samples = signal.shape[-1]
    t = np.arange(num_samples) / fs
    doppler_phase = np.exp(1j * 2 * np.pi * fd * t)
    return signal * doppler_phase

# === SNR测量函数 ===
def measure_snr(clean_signal, noisy_signal):
    """
    测量实际SNR
    """
    signal_power = np.mean(np.abs(clean_signal) ** 2)
    noise = noisy_signal - clean_signal
    noise_power = np.mean(np.abs(noise) ** 2)
    
    if noise_power == 0:
        return float('inf')
    
    snr_measured = 10 * np.log10(signal_power / noise_power)
    return snr_measured

# === 复数AWGN噪声添加函数 ===
def add_complex_awgn(signal, snr_db):
    """
    为复数信号添加AWGN噪声
    """
    # 计算信号功率
    signal_power = np.mean(np.abs(signal) ** 2)
    
    # 计算噪声功率
    snr_linear = 10 ** (snr_db / 10)
    noise_power = signal_power / snr_linear
    
    # 生成复数噪声（实部和虚部独立，各占一半功率）
    noise_std = np.sqrt(noise_power / 2)
    noise_real = np.random.normal(0, noise_std, signal.shape)
    noise_imag = np.random.normal(0, noise_std, signal.shape)
    noise = noise_real + 1j * noise_imag
    
    return signal + noise, noise

# === 带验证的预处理函数 ===
def preprocess_iq_data(data_real_imag, snr_db=None, fd=None, fs=None, 
                       add_noise=True, add_doppler=True, verify_snr=True):
    """
    预处理IQ数据：可选择性地添加噪声和多普勒频移
    
    参数:
    - data_real_imag: 输入数据，shape (N, T, 2)
    - snr_db: 目标信噪比(dB)，当add_noise=True时必需
    - fd: 多普勒频移(Hz)，当add_doppler=True时必需
    - fs: 采样率(Hz)，当add_doppler=True时必需
    - add_noise: 是否添加噪声 (默认True)
    - add_doppler: 是否添加多普勒频移 (默认True)
    - verify_snr: 是否验证SNR (仅当add_noise=True时有效)
    
    返回:
    - processed_real_imag: 处理后的数据，shape (N, T, 2)
    - snr_info: SNR验证信息（如果verify_snr=True且add_noise=True）
    """
    # 参数检查
    if add_noise and snr_db is None:
        raise ValueError("当add_noise=True时，必须提供snr_db参数")
    
    if add_doppler and (fd is None or fs is None):
        raise ValueError("当add_doppler=True时，必须提供fd和fs参数")
    
    # Step 1: 转为复数 IQ，shape: (N, T, 2) → (N, T)
    data_complex = data_real_imag[..., 0] + 1j * data_real_imag[..., 1]

    processed = []
    snr_measured_list = []
    
    for i, sig in enumerate(data_complex):
        current_sig = sig.copy()
        
        # Step 2: 添加 AWGN 噪声（可选）
        if add_noise:
            noisy_sig, noise = add_complex_awgn(current_sig, snr_db)
            current_sig = noisy_sig
            
            # SNR验证（可选）
            if verify_snr:
                measured_snr = measure_snr(sig, noisy_sig)
                snr_measured_list.append(measured_snr)
                
                # 每10000个样本打印一次进度
                if i % 10000 == 0 and i > 0:
                    avg_snr = np.mean(snr_measured_list[-10000:])
                    print(f"[验证] 样本 {i}, 平均实测SNR: {avg_snr:.2f} dB")
        
        # Step 3: 添加多普勒频移（可选）
        if add_doppler:
            shifted = add_doppler_shift(current_sig, fd, fs)
            current_sig = shifted
            
        processed.append(current_sig)

    processed = np.array(processed)
    
    # Step 4: 转回 [I, Q] 实数格式
    processed_real_imag = np.stack([processed.real, processed.imag], axis=-1)
    
    # SNR验证总结（仅当添加噪声且启用验证时）
    snr_info = None
    if add_noise and verify_snr and snr_measured_list:
        snr_measured_array = np.array(snr_measured_list)
        snr_info = {
            'target_snr': snr_db,
            'measured_mean': np.mean(snr_measured_array),
            'measured_std': np.std(snr_measured_array),
            'measured_min': np.min(snr_measured_array),
            'measured_max': np.max(snr_measured_array),
            'samples_count': len(snr_measured_array)
        }
        
        print(f"\n=== SNR验证结果 ===")
        print(f"目标SNR: {snr_info['target_snr']} dB")
        print(f"实测平均SNR: {snr_info['measured_mean']:.2f} dB")
        print(f"实测标准差: {snr_info['measured_std']:.2f} dB")
        print(f"实测范围: [{snr_info['measured_min']:.2f}, {snr_info['measured_max']:.2f}] dB")
        print(f"验证样本数: {snr_info['samples_count']}")
    
    return processed_real_imag, snr_info

# === 单样本测试函数 ===
def test_single_sample_snr(data_real_imag, snr_db, num_tests=10):
    """
    对单个样本进行多次SNR测试，确保噪声添加的正确性
    """
    print(f"\n=== 单样本SNR测试 (测试{num_tests}次) ===")
    
    # 取第一个样本
    sample_complex = data_real_imag[0, :, 0] + 1j * data_real_imag[0, :, 1]
    
    measured_snrs = []
    for i in range(num_tests):
        noisy_sample, _ = add_complex_awgn(sample_complex, snr_db)
        measured_snr = measure_snr(sample_complex, noisy_sample)
        measured_snrs.append(measured_snr)
        print(f"测试 {i+1}: 目标SNR={snr_db} dB, 实测SNR={measured_snr:.2f} dB")
    
    measured_snrs = np.array(measured_snrs)
    print(f"\n测试统计:")
    print(f"平均值: {np.mean(measured_snrs):.2f} dB")
    print(f"标准差: {np.std(measured_snrs):.2f} dB")
    print(f"误差范围: ±{np.abs(np.mean(measured_snrs) - snr_db):.2f} dB")

# === 使用示例 ===
if __name__ == "__main__":
    # 假设 X_train, X_test 已定义
    
    # 使用全局变量控制处理选项
    X_train_processed, _ = preprocess_iq_data(
        X_train, snr_db=SNR_dB, fd=fd, fs=fs, 
        add_noise=Add_noise, add_doppler=Add_doppler, verify_snr=False
    )
    
    # 单样本测试（只有当Add_noise=True时才有意义）
    if Add_noise:
        test_single_sample_snr(X_train, SNR_dB, num_tests=5)
    else:
        print("\n=== 跳过单样本SNR测试（未启用噪声添加）===")
    
    # 处理测试集（可以根据需要设置不同的选项）
    print(f"\n=== 处理测试集 ===")
    X_test_processed, test_snr_info = preprocess_iq_data(
        X_test, snr_db=SNR_dB, fd=fd, fs=fs, 
        add_noise=Add_noise, add_doppler=Add_doppler, verify_snr=False
    )
    
    # 查看处理前后前10个点
    print(f"\n=== 信号对比 ===")
    print("原始信号 I 分量：", X_train[0, :10, 0])
    print("处理后信号 I 分量：", X_train_processed[0, :10, 0])
    
    # 验证多普勒频移效果（只有当Add_doppler=True时才有意义）
    if Add_doppler:
        print(f"\n=== 多普勒频移验证 ===")
        original_phase = np.angle(X_train[0, :10, 0] + 1j * X_train[0, :10, 1])
        processed_phase = np.angle(X_train_processed[0, :10, 0] + 1j * X_train_processed[0, :10, 1])
        phase_diff = processed_phase - original_phase
        print("相位变化:", phase_diff)
    else:
        print(f"\n=== 跳过多普勒频移验证（未启用多普勒频移）===")
import os
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
from torch.utils.data import DataLoader, TensorDataset, Subset
from datetime import datetime
from tqdm import tqdm
from sklearn.model_selection import KFold
from torch.nn import TransformerEncoder, TransformerEncoderLayer

# 假设 SNR_dB 和 fd 已经定义
SNR_dB = globals().get('SNR_dB', 'no')
fd = globals().get('fd', 'no')

# === 模型与训练参数设置 ===
raw_input_dim = 2         # 每个时间步是 I/Q 两个值
model_dim = 128           # Transformer 模型内部维度
num_heads = 4
num_layers = 3
num_classes = len(np.unique(y_train))  # 或 len(tx_list)
dropout = 0.1
batch_size = 512
num_epochs = 200
learning_rate = 1e-4
patience = 5

# === 创建保存目录 ===
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
script_name = "wisig_time"
folder_name = f"{timestamp}_{script_name}_SNR{SNR_dB}dB_fd{fd}_classes_{num_classes}_Transformer"
save_folder = os.path.join(os.getcwd(), "training_results", folder_name)
os.makedirs(save_folder, exist_ok=True)

results_file = os.path.join(save_folder, "results.txt")
with open(results_file, "w") as f:
    f.write(f"=== Experiment Summary ===\n")
    f.write(f"Timestamp: {timestamp}\n")
    f.write(f"Total Classes: {num_classes}\n")
    f.write(f"SNR: {SNR_dB} dB\n")
    f.write(f"fd (Doppler shift): {fd} Hz\n")
    f.write(f"equalized: {equalized}\n")

# === 模型定义 ===
class SignalTransformer(nn.Module):
    def __init__(self, raw_input_dim, model_dim, num_heads, num_layers, num_classes, dropout=0.1):
        super(SignalTransformer, self).__init__()
        self.embedding = nn.Linear(raw_input_dim, model_dim)
        encoder_layer = TransformerEncoderLayer(d_model=model_dim, nhead=num_heads, dropout=dropout, batch_first=True)
        self.encoder = TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(model_dim, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = self.encoder(x)
        x = x[:, -1, :]
        x = self.fc(x)
        return x


# === 假设 X_train, y_train, X_test, y_test 都已定义并 shape 为 (N, L, 2) ===
# 若还未定义，可自行加载并 reshape
X_test_tensor = torch.tensor(X_test_processed, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

train_dataset = TensorDataset(torch.tensor(X_train_processed, dtype=torch.float32),
                               torch.tensor(y_train, dtype=torch.long))

# === K折交叉验证训练 ===
n_splits = 5
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
fold_results = []
test_results = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def compute_grad_norm(model):
    total_norm = 0.0
    for p in model.parameters():
        if p.grad is not None:
            param_norm = p.grad.data.norm(2)
            total_norm += param_norm.item() ** 2
    return total_norm ** 0.5

def moving_average(x, w=5):
    return np.convolve(x, np.ones(w), 'valid') / w

avg_grad_norms_per_fold = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(train_dataset)):
    print(f"\n====== Fold {fold+1}/{n_splits} ======")

    train_subset = Subset(train_dataset, train_idx)
    val_subset = Subset(train_dataset, val_idx)

    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, drop_last=True)

    model = SignalTransformer(raw_input_dim, model_dim, num_heads, num_layers, num_classes, dropout).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    grad_norms = []

    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(num_epochs):
        model.train()
        running_train_loss, correct_train, total_train = 0.0, 0, 0
        batch_grad_norms = []

        with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as tepoch:
            for inputs, labels in tepoch:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()

                grad_norm = compute_grad_norm(model)
                batch_grad_norms.append(grad_norm)

                optimizer.step()

                running_train_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total_train += labels.size(0)
                correct_train += (predicted == labels).sum().item()

                tepoch.set_postfix(loss=running_train_loss / (len(train_loader)),
                                   accuracy=100 * correct_train / total_train,
                                   grad_norm=grad_norm)

        epoch_train_loss = running_train_loss / len(train_loader)
        train_losses.append(epoch_train_loss)
        train_accuracies.append(100 * correct_train / total_train)
        avg_grad_norm = np.mean(batch_grad_norms)
        grad_norms.append(avg_grad_norm)

        print(f"Epoch {epoch+1} Average Gradient Norm: {avg_grad_norm:.4f}")

        # === 验证 ===
        model.eval()
        running_val_loss, correct_val, total_val = 0.0, 0, 0

        with torch.no_grad():
            for val_inputs, val_labels in val_loader:
                val_inputs = val_inputs.to(device)
                val_labels = val_labels.to(device)

                val_outputs = model(val_inputs)
                val_loss = criterion(val_outputs, val_labels)
                running_val_loss += val_loss.item()
                _, val_predicted = torch.max(val_outputs, 1)
                total_val += val_labels.size(0)
                correct_val += (val_predicted == val_labels).sum().item()

        epoch_val_loss = running_val_loss / len(val_loader)
        val_losses.append(epoch_val_loss)
        val_accuracies.append(100 * correct_val / total_val)

        with open(results_file, "a") as f:
            f.write(f"Epoch {epoch+1} | Train Acc: {train_accuracies[-1]:.2f}% | Val Acc: {val_accuracies[-1]:.2f}%\n")

        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print("Early stopping")
            break

        scheduler.step()

    fold_results.append(max(val_accuracies))
    avg_grad_norms_per_fold.append(grad_norms)

    # === 绘制 loss 曲线 ===
    plt.figure()
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Val Loss')
    plt.plot(moving_average(train_losses), label='Train Loss (Smooth)', linestyle='--')
    plt.plot(moving_average(val_losses), label='Val Loss (Smooth)', linestyle='--')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(f'Fold {fold+1} Loss Curve')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(save_folder, f"fold_{fold+1}_loss_curve.png"))
    plt.close()

    # === 绘制 Gradient Norm 曲线 ===
    plt.figure()
    plt.plot(grad_norms, label='Gradient Norm')
    plt.xlabel('Epoch')
    plt.ylabel('Gradient Norm')
    plt.title(f'Fold {fold+1} Gradient Norm')
    plt.grid(True)
    plt.legend()
    plt.savefig(os.path.join(save_folder, f"fold_{fold+1}_grad_norm.png"))
    plt.close()

    # === 测试集评估 ===
    model.eval()
    test_preds, test_true = [], []

    with torch.no_grad():
        for test_inputs, test_labels in test_loader:
            test_inputs = test_inputs.to(device)
            test_labels = test_labels.to(device)

            test_outputs = model(test_inputs)
            _, predicted = torch.max(test_outputs, 1)
            test_preds.extend(predicted.cpu().numpy())
            test_true.extend(test_labels.cpu().numpy())

    test_preds = np.array(test_preds)
    test_true = np.array(test_true)
    test_accuracy = 100.0 * np.sum(test_preds == test_true) / len(test_true)
    test_results.append(test_accuracy)

    with open(results_file, "a") as f:
        f.write(f"Fold {fold+1} Test Accuracy: {test_accuracy:.2f}%\n")

    cm = confusion_matrix(test_true, test_preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Test Confusion Matrix Fold {fold+1}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig(os.path.join(save_folder, f"fold_{fold+1}_test_confusion_matrix.png"))
    plt.close()

# === 总结结果 ===
avg_val = np.mean(fold_results)
avg_test = np.mean(test_results)

with open(results_file, "a") as f:
    f.write("\n=== Summary ===\n")
    for i in range(n_splits):
        f.write(f"Fold {i+1}: Val Acc = {fold_results[i]:.2f}%, Test Acc = {test_results[i]:.2f}%\n")
    f.write(f"\nAverage Validation Accuracy: {avg_val:.2f}%\n")
    f.write(f"Average Test Accuracy: {avg_test:.2f}%\n")

print("\n=== Final Summary ===")
for i in range(n_splits):
    print(f"Fold {i+1}: Val = {fold_results[i]:.2f}%, Test = {test_results[i]:.2f}%")
print(f"Average Val Accuracy: {avg_val:.2f}%")
print(f"Average Test Accuracy: {avg_test:.2f}%")


In [ ]:
# 参数搜索
import os
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
from torch.utils.data import DataLoader, TensorDataset, random_split
from datetime import datetime
from tqdm import tqdm
import random
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from sklearn.model_selection import train_test_split

# 假设 SNR_dB, fd, equalized 已定义
SNR_dB = globals().get('SNR_dB', 'no')
fd = globals().get('fd', 'no')
equalized = globals().get('equalized', 'no')

# 假设 X_train_processed, y_train, X_test_processed, y_test 已定义
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# === 模型定义 ===
class SignalTransformer(nn.Module):
    def __init__(self, raw_input_dim, model_dim, num_heads, num_layers, num_classes, dropout=0.1):
        super(SignalTransformer, self).__init__()
        self.embedding = nn.Linear(raw_input_dim, model_dim)
        encoder_layer = TransformerEncoderLayer(
            d_model=model_dim, nhead=num_heads, dropout=dropout, batch_first=True
        )
        self.encoder = TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(model_dim, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = self.encoder(x)
        x = x[:, -1, :]
        x = self.fc(x)
        return x

# === 数据准备 ===
X_train_tensor = torch.tensor(X_train_processed, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_processed, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

num_classes = len(np.unique(y_train))

# 划分训练集 / 验证集
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# === 参数空间 ===
param_space = {
    "model_dim": [128, 256, 512],
    "num_heads": [2, 4, 8],
    "num_layers": [1, 2, 3],
    "dropout": [0.1, 0.3, 0.5],
    "learning_rate": [1e-3, 5e-4, 1e-4],
    "batch_size": [128, 256, 512]
}
num_search = 50  # 随机搜索次数
patience = 5
raw_input_dim = 2
num_epochs = 300

results_summary = []
best_config = None
best_val_acc = 0

# 计算梯度范数
def compute_grad_norm(model):
    total_norm = 0.0
    for p in model.parameters():
        if p.grad is not None:
            param_norm = p.grad.data.norm(2)
            total_norm += param_norm.item() ** 2
    return total_norm ** 0.5

# 平滑曲线
def moving_average(x, w=5):
    return np.convolve(x, np.ones(w), 'valid') / w

# === 随机搜索 ===
for search_idx in range(num_search):
    config = {k: random.choice(v) for k, v in param_space.items()}
    print(f"\n=== Random Search {search_idx+1}/{num_search} ===")
    print(f"Params: {config}")

    # 创建保存目录
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    script_name = "wisig_time_random"
    folder_name = f"{timestamp}_{script_name}_SNR{SNR_dB}"
    save_folder = os.path.join("search_results", folder_name)
    os.makedirs(save_folder, exist_ok=True)
    results_file = os.path.join(save_folder, "results.txt")

    with open(results_file, "w") as f:
        f.write(f"=== Hyperparameters ===\n{config}\n")

    # DataLoader
    train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True, drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=config["batch_size"], shuffle=False, drop_last=True)
    test_loader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False)

    # 模型 & 优化器
    model = SignalTransformer(raw_input_dim, config["model_dim"], config["num_heads"],
                              config["num_layers"], num_classes, config["dropout"]).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"], weight_decay=1e-4)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    grad_norms = []

    best_val = 0
    patience_counter = 0
    best_model_wts = None

    for epoch in range(num_epochs):
    # 训练
        model.train()
        running_loss, correct_train, total_train = 0.0, 0, 0
        batch_grad_norms = []

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            grad_norm = compute_grad_norm(model)
            batch_grad_norms.append(grad_norm)
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        train_losses.append(running_loss / len(train_loader))
        train_accuracies.append(100 * correct_train / total_train)
        grad_norms.append(np.mean(batch_grad_norms))

        # 验证
        model.eval()
        correct_val, total_val = 0, 0
        val_loss_sum = 0.0
        with torch.no_grad():
            for val_inputs, val_labels in val_loader:
                val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
                val_outputs = model(val_inputs)
                val_loss = criterion(val_outputs, val_labels)
                val_loss_sum += val_loss.item()
                _, val_pred = torch.max(val_outputs, 1)
                total_val += val_labels.size(0)
                correct_val += (val_pred == val_labels).sum().item()

        val_acc = 100 * correct_val / total_val
        val_losses.append(val_loss_sum / len(val_loader))
        val_accuracies.append(val_acc)

        # 早停
        if val_acc > best_val:
            best_val = val_acc
            patience_counter = 0
            best_model_wts = model.state_dict()
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

        scheduler.step()


    # 恢复最佳权重
    if best_model_wts:
        model.load_state_dict(best_model_wts)

    # 测试集
    model.eval()
    test_preds, test_true = [], []
    with torch.no_grad():
        for test_inputs, test_labels in test_loader:
            test_inputs, test_labels = test_inputs.to(device), test_labels.to(device)
            test_outputs = model(test_inputs)
            _, predicted = torch.max(test_outputs, 1)
            test_preds.extend(predicted.cpu().numpy())
            test_true.extend(test_labels.cpu().numpy())

    test_acc = 100 * np.sum(np.array(test_preds) == np.array(test_true)) / len(test_true)
    with open(results_file, "a") as f:
        f.write(f"\nVal Acc: {val_acc:.2f}% | Test Acc: {test_acc:.2f}%\n")

    # 控制台即时输出
    print(f"[Result] Config {search_idx+1}/{num_search} - Val Acc: {val_acc:.2f}%, Test Acc: {test_acc:.2f}%")

    # 记录结果
    results_summary.append((config, best_val, test_acc))
    if best_val > best_val_acc:
        best_val_acc = best_val
        best_config = (config, test_acc)


# === 最佳结果 ===
print("\n=== Best Hyperparameters ===")
print(best_config)
